## Autoencoder in Tensorflow
Implementation of a symetrical deep autoencoder with weight-tying using keras

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

## AutoencoderBuilder
The builder class abstracts away the process of building a multi-layered, symmetrical tied-weight autoencoder

In [0]:
class AutoencoderBuilder():
  """
  Builder for a multi-layer, symetrical autoencoder with weight-tying
  """
  def __init__(self, input_shape, encoder_layer_sizes, activation='relu'):
    """
    The decoder layer sizes will be symetrical of the encoder. The last layer
    of the encoding layer describes the bottle-neck layer or the latent 
    representation of the input. Note: the input data must be normalized. 
    """
    self.input_shape = input_shape
    self.encoder_layer_sizes = encoder_layer_sizes
    self.activation = activation
    
  def build(self):
    """Builds the autoencoder"""
    encoder_layers = self._encoder()
    decoder_layers = self._decoder(encoder_layers)
    return keras.models.Sequential(encoder_layers + decoder_layers)

  def _encoder(self):
    """Builds the encoding layers"""
    layers = []
    layers.append(keras.layers.Flatten(input_shape=(self.input_shape)))
    for size in self.encoder_layer_sizes:
      layers.append(keras.layers.Dense(size, activation=self.activation))
    return layers

  def _decoder(self, encoder_layers):
    """
    Builds the decoding layers by transposing encoder layers such
    that the encoder and decoder share weights
    """
    layers = []
    for el in encoder_layers[:1:-1]:
      layers.append(DenseTranspose(el, activation=self.activation))
    layers.append(DenseTranspose(encoder_layers[1], activation='sigmoid'))
    layers.append(keras.layers.Reshape(self.input_shape))
    return layers

## Custom weight-tying layers

In [0]:
# weight-tying dense (transpose) layer
# cite: https://mc.ai/a-beginners-guide-to-build-stacked-autoencoder-and-tying-weights-with-it/

class DenseTranspose(keras.layers.Layer):
  def __init__(self, dense, activation=None, **kwargs):
    super().__init__(**kwargs)
    self.dense = dense
    self.activation = keras.activations.get(activation)
  
  def build(self, input_shape):
    super().build(input_shape)
    self.bias = self.add_weight(name="bias",
                                shape=(self.dense.input_shape[-1]),
                                initializer="zeros")
  
  def call(self, inputs):
    z = tf.matmul(inputs, self.dense.weights[0], transpose_b=True)
    return self.activation(z + self.bias)

## Testing on MNIST dataset

#### MNIST data loading and preprocessing

In [4]:
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# normalizing the data
x_train, x_test = x_train / 255.0, x_test /255.0

11493376/11490434 [==============================] - 0s 0us/step


#### Building an autoencoder

In [5]:
ae_builder = AutoencoderBuilder(input_shape=(28,28),
                                encoder_layer_sizes=[392, 196],
                                activation='relu')
ae = ae_builder.build()
ae.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 392)               307720    
_________________________________________________________________
dense_1 (Dense)              (None, 196)               77028     
_________________________________________________________________
dense_transpose (DenseTransp (None, 392)               77420     
_________________________________________________________________
dense_transpose_1 (DenseTran (None, 784)               308504    
_________________________________________________________________
reshape (Reshape)            (None, 28, 28)            0         
Total params: 385,924
Trainable params: 385,924
Non-trainable params: 0
__________________________________________________

#### Model compiling and traning

In [6]:
ae.compile(optimizer='adam',
           loss='mean_squared_error',
           metrics=['accuracy'])
ae.fit(x_train, x_train, epochs=20)

Epoch 1/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0106 - accuracy: 0.2846
Epoch 2/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0038 - accuracy: 0.3240
Epoch 3/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0030 - accuracy: 0.3341
Epoch 4/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0025 - accuracy: 0.3397
Epoch 5/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0023 - accuracy: 0.3437
Epoch 6/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0022 - accuracy: 0.3469
Epoch 7/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0020 - accuracy: 0.3493
Epoch 8/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0019 - accuracy: 0.3515
Epoch 9/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0018 - accuracy: 0.3532
Epoch 10/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.001

### TODO: extract trained encoder for dimension reduction